# Subtask 1:
## Statistics

In [ ]:
import pandas as pd         # data handling
#import matplotlib           # histogram plotting
import matplotlib.pyplot as plt

candidatesFileName = 'files/candidates_V2.csv'
#candidatesnp = np.genfromtxt('.candidates_V2.csv', delimiter=',', dtype=None, skip_header=0)
candidates = pd.read_csv(candidatesFileName)

candidates

In [ ]:
print("The total number of scans is " + str(len(candidates)) + ".")
print("There are " + str(candidates['seriesuid'].nunique()) + " different UIDs.")
nNonnodules = str(candidates['class'].value_counts()[0])
nNodules = str(candidates['class'].value_counts()[1])
print("There are " + nNonnodules + " non nodules and " +
      nNodules + " nodules.")

counts = candidates.groupby(['seriesuid', 'class']).size().unstack(fill_value=0)
#counts.hist(legend=True)

fig, axes = plt.subplots(1, 2, sharey=True)
counts[0].hist( ax=axes[0])
axes[0].set_title('Non-nodules per unique scan')
axes[0].set_xlabel('Number of non-nodules items')
axes[0].set_ylabel('Number of scans')

counts[1].hist( ax=axes[1])
axes[1].set_title('Nodules per scan')
axes[1].set_xlabel('Number of nodules items')

plt.tight_layout()
plt.show()

## Loading

In [ ]:
import numpy as np
import itk
import matplotlib.pyplot as plt
import os

scan_path = 'files/subset/scans/'
segment_path = "files/subset/segments/"
savepath = "files/extracted/"
listing = os.listdir(scan_path)
mhdfiles = [fname for fname in listing if fname.endswith('.mhd')]

for uid in mhdfiles:
    mhd_data = itk.imread(scan_path + uid)
    try:
        mhd_data_segment = itk.imread(segment_path + uid)
    except:
        print("File doesn't exist")
        continue

    # convert to numpy array
    scan = np.array(itk.GetArrayFromImage(mhd_data), dtype=np.float32)
    scan_segment = np.array(itk.GetArrayFromImage(mhd_data_segment), dtype=np.float32)
    clip_win = (-1000, 1000)
    scan.clip(clip_win[0], clip_win[1], scan)  # clip to reasonable HU values

    origin_xyz = np.asarray(mhd_data.GetOrigin())
    voxel_size_xyz = np.asarray(mhd_data.GetSpacing())
    #voxel_size_xyz = mhd_data.GetSpacing()
    direction_matrix = np.asarray(mhd_data.GetDirection()).reshape(3, 3) # changed to asarray from array to fix type error

    matchingc = candidates[candidates['seriesuid'] == uid[:-4]]
    c = {0: 'b', 1: 'r'} # dict

    for (_,i) in matchingc.iterrows():
        # Get world coordinates
        world_coords = i.loc[['coordX', 'coordY', 'coordZ']].to_numpy(dtype=np.float32)
        # Transformation into voxel ("pixel") coordinates
        voxel_coords = np.linalg.inv(direction_matrix) @ ((world_coords - origin_xyz)/voxel_size_xyz)
        voxel_coords_rounded = np.int16(voxel_coords.round(0))

        fig, (ax1, ax2, ax3) = plt.subplots(1,3)


        # Show full image, nod/nonnod in circle
        plt.subplot(1,3,1)
        plt.imshow(scan[voxel_coords_rounded[2], :, :], cmap=plt.cm.gray)
        nod_coords = (voxel_coords_rounded[0], voxel_coords_rounded[1])
        ax1.add_patch(plt.Circle(nod_coords, 10, color=c[i["class"]], fill=False))

        # Show segment
        plt.subplot(1,3,2)
        slice = scan[voxel_coords_rounded[2], :, :]
        
        # Extract segment
        slicebox = np.array([(nod_coords[1]-32), (nod_coords[1]+32), (nod_coords[0]-32), (nod_coords[0]+32)])
        if (np.any(slicebox < 0) or np.any(slicebox > 512)):
            continue
        extracted_patch = slice[(nod_coords[1]-32):(nod_coords[1]+32), (nod_coords[0]-32):(nod_coords[0]+32)]
        plt.imshow(extracted_patch, cmap=plt.cm.gray, vmin=clip_win[0], vmax=clip_win[1])
        #plt.savefig('output.png')
        
        plt.imsave(f'{savepath}{i["class"]}/{uid[:-4]}_{voxel_coords_rounded[2]}.png', extracted_patch, cmap=plt.cm.gray, vmin=clip_win[0], vmax=clip_win[1])
        
        # Show segmented imageR
        plt.subplot(1,3,3)
        plt.imshow(scan_segment[voxel_coords_rounded[2], :, :], cmap=plt.cm.nipy_spectral)
